# Truebot
> Truely a simple malware leading to ransomware

- toc: true 
- badges: true
- categories: [truebot,config,triage]

## Overview

Truebot (aka Silence) is primarily a downloader associated with the threat actor group [TA505](https://malpedia.caad.fkie.fraunhofer.de/actor/ta505). Recently there was a CISA alert [Increased Truebot Activity Infects U.S. and Canada Based Networks](https://www.cisa.gov/news-events/cybersecurity-advisories/aa23-187a) which described ransomware/extortion activity associated with the use of Truebot.

### References
- [Increased Truebot Activity Infects U.S. and Canada Based Networks](https://www.cisa.gov/news-events/cybersecurity-advisories/aa23-187a) 
- [TrueBot Analysis Part III - Capabilities](https://malware.love/malware_analysis/reverse_engineering/2023/03/31/analyzing-truebot-capabilities.html)
- [10445155.r1.v1 (pdf)](https://www.cisa.gov/sites/default/files/2023-07/MAR-10445155.r1.v1.CLEAR_.pdf)
- [A Truly Graceful Wipe Out](https://thedfirreport.com/2023/06/12/a-truly-graceful-wipe-out/)

### Samples
- [717beedcd2431785a0f59d194e47970e9544fbf398d462a305f6ad9a1b1100cb](https://www.unpac.me/results/8d1eb4c3-cbb0-4b32-8d52-6f142c836d0f?hash=717beedcd2431785a0f59d194e47970e9544fbf398d462a305f6ad9a1b1100cb#/)

## Analysis

- The binary uses an Adobe PDF icon possibly to trick victims into clicking it. It also displays a fake error message. `"There was an error opening this document. The file is damaged and could not be repaired. Adobe Acrobat"`
- The binary is padded with a significant amount of junk code that is not relevant to its operation. 
- The main code has some checks for debugging tools and AV which if detected cause the malware to execute a deception process (calc.exe)
- There are multiple anti-emulation techniques used including ...
  - Reading from a fake named pipe
  - Calling EraseTape
  - Checking for a valid code page with GetACP
  - Loading `user32`
  - Trying to open a random invalid file
- The C2 host and URL path are encrypted using base64, urldecode, and RC4 with a hard coded key
  - `essadonio.com`
  - `/538332[.]php`
- A hard coded mutex `OrionStartWorld#666` is used to ensure only one copy of the malware is running 
- A GUID is generated for the victim and stored in a randomly named file with the extension `.JSONMSDN` in the `%APPDATA%` directory. This GUID is also used in the C2 communications.
- A list of processes running on the host is collected and combined with the GUID. It is base64 encoded then sent to the C2 server in a POST request.
- The C2 has the option of sending the following commands
  - **LSEL** - delete yourself and exit
  - **TFOUN** - array of commands
    - **EFE** - download payload, decrypt with RC4 (hard coded key), and execute PE
    - **S66** - download, decrypt with RC4 (hard coded key),and inject shellcode into `cmd.exe`
    - **Z66** - download, decrypt with RC4 (hard coded key),and run shellcode 


